# Generating Strings with Metropolis Hastings

In this notebook, we will: 

1. Read a long book and compute 3-gram frequencies (i.e. frequencies with which three letter combinations occurs in words).

2. Given the 3-gram frequency distribution, we will use the Metropolis Hastings algorithm to generate long strings.

In [23]:
from tqdm.notebook import tqdm, trange
import multiprocessing as mp
from itertools import product
import numpy as np
        
rng = np.random.default_rng( np.random.SFC64() )

In [24]:
# Helper functions to go between a list of indexes to strings.
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
index = {s:n for n, s in enumerate(alphabet)}
stringify = lambda X: ''.join( alphabet[x] for x in X )
tolist = lambda s: [ index[c] for c in s ]


The 3-gram frequencies were generated using a few books, and are stored in the `frequencies.npz` data file.

`F` is a `26 x 26 x 26` array, where `F[i,j,k]` is the actual frequency with which the string containing the $i^\text{th}$, $j^\text{th}$ and $k^\text{th}$ symbols of the alphabet (in that order) occurred. Note that $i, j, k \in \{ 0, \dots, 25 \}$.

In [25]:
npz = np.load( 'frequencies.npz' )

F = npz['F']
ngram = len(F.shape)

## Algorithm

Let $N \in \mathbb{N}$, and let $\mathcal X$ be the set of all length $N$ sequences of letters in the alphabet.
If $x = (x_1, x_2, \dots, x_N) \in \mathcal X$, define
$$
  \pi_u(x) = \prod_{i=1}^{N-2} (1 + F( x_i, x_{i+1}, x_{i+2} ))
$$

We will use the Metropolis Hastings algorithm to sample points from $\mathcal X$ with probability proportional to $\pi_u$.
For convenience, here is a python implementation of the function $\pi_u$. In this implementation, `x` is a list (or array), and `x[i]` is an integer between $0$ and $25$ reprenting the index of the corresponding symbol in the alphabet.

In order to deal with overflows for longer strings, we compute sum of the logs of the expression above and exponentiate as follows:

$$
  \pi_u(x) = \exp(\sum_{i=1}^{N-2} (1 + F( x_i, x_{i+1}, x_{i+2} )))
$$


In [26]:
F1 = F + 1
πu = lambda x: sum ([np.log(F1[tuple(x[i:i+ngram])]) for i in range(0, len(x)-ngram+1) ] )

Given $x = (x_1, \dots, x_N) \in \mathcal X$, we propose a state $y = (y_1, \dots, y_N)$ as follows:

1. First choose $i \in \{1, \dots, N \}$ uniformly at random.

2. If $i \leq 2$, choose $y_i'$ from the alphabet uniformly at random.

3. If $i > 2$, choose $y_i'$ from the alphabet so that $\mathbf P( y_i' = y ) \propto 1 + F(x_{i-2}, x_{i-1}, y )$.

4. Define the new state $y = (y_1, \dots, y_N)$ by
  \begin{equation}
    y_j =
      \begin{cases}
        x_j & j \neq i\,,\\
        y_i' & j = i \,.
      \end{cases}
  \end{equation}



We define the acceptance ratio for this proposal mechanism as $A(x, y)$ and $Q(x,y)$ as the probability of accepting the transition of $x$ to $y$. An extra parameter `pos` is included, which represents the index at which $x$ and $y$  differ.

In [27]:
def Q(x,y, pos):
    # return the frequency of the 3-gram with the new updated string
    return F1[x[pos-2], x[pos-1], y[pos]]
def A(x, y, pos):
    if pos < 2:
        # return the difference in the values of π for x and y
        return min(1, np.exp(πu(y) - πu(x)))
    # else, compute π and Q and use the standard Metroplis Hastings Acceptance Ratio
    return min(1, np.exp(πu(y)-πu(x)) * (Q(y,x,pos)/Q(x,y,pos)))
    

##### Testing the Accpetance Ratio Implementation

The output of both cells below should be the same (and be around  
0.046) since the changed index occurs around the same indexes for the same 3-grams:

In [28]:
changed_index = lambda x, y: next( i for i in range( len(x) ) if x[i] != y[i] )
x, y = (tolist(s) for s in ('OVERTHE', 'ODERTHE') )
A(x, y, changed_index(x, y) )

0.04601938930928614

In [29]:
x, y = (tolist(s) for s in (
    'OLDMACDONALDHADAFARMEIEIOTHEQUICKBROWNFOXJUMPSOVERTHELAZYDOG',
    'OLDMACDONALDHADAFARMEIEIOTHEQUICKBROWNFOXJUMPSODERTHELAZYDOG') )
A(x, y, changed_index(x, y) )

0.04601938930928629

### Implementing Metrpolis Hastings Sampling

The function `MetropolisHastings` implements the proposal mechanism described earlier, choosing the accept a new state based on the acceptance ratio:

In [34]:

def MetropolisHastings( x, n_iters ):
    y = x.copy()
    alphaList = tolist(alphabet)
    for _ in range(n_iters):
        y_prime = y.copy()
        i = np.random.randint(0, len(x)-1)
        if i < 2:
            y_prime[i] = rng.choice(alphaList)
        else:
            y_prime[i] = rng.choice(alphaList, p=F1[y[i-2],y[i-1]]/sum(F1[y[i-2],y[i-1]]))
        accept = A(y, y_prime, i)
        if(accept == 1 or np.random.binomial(1, accept)):
            y = y_prime.copy()
    return y


### Final Tests

We generate long strings and print the current state at every few iterations. A higher value for $πu$ is better and tells us that each substring of length 3 appears frequently in our sample texts. In general, our algorithm increases the value of $πu$ on each iteration and generates words in the English dictionary!

In [37]:
%%time
X = rng.integers( len(alphabet), size=60 ) # Start randomly
print( f'{0:3} {stringify(X)}, πu={πu(X):.2f}' )
for n in range(20):
    X = MetropolisHastings( X, 1000 )
    print( f'{n+1:3} {stringify(X)}, πu={πu(X):.2f}' )

  0 KYDPGAGPUAFXQAAPSUBEOVPZIVDYOVMWVKDRDGBVZFVFYTVIPPERLWVVESDE, πu=49.79
  1 EMERVERESTEEQUESHOMETHITEROPERENTIONESSULAINDECARRINTEETENBE, πu=360.52
  2 AMEATERINITEQUATHALITHATHROPEREATHERESTANDISSECASTINDINTERSE, πu=375.26
  3 GLINTERATEREQUESTINATHEREMEMENDITHEREADINDISTERMOVERMINDENTE, πu=382.01
  4 ENESTERATEREQUENTICARTIMESINSTINTEARRENINSINGANDEREALINNINDE, πu=367.60
  5 ELENSIDATHEEQUELIETINDEPARTHILENTHASTITARPENTATHERETHINTENTE, πu=365.20
  6 SHANCANTEREEQUILINTENTEREATHERINDEOUTEVEREARAITHALATHALLENTE, πu=380.50
  7 THINTINDINDEQUALLOVERIEVERTHERINGLINTENOSINDEATHARETHALLEATE, πu=383.47
  8 THEATINTELEEQUALLACOREAREATHERINTIENTERIOUNDEATHERATHALEANTE, πu=388.76
  9 THEATINTEREEQUEREARATHEREATHERATHEANDENSORSTANCHARITHARIANGE, πu=384.47
 10 THALLANDEREEQUERAPEATHEMISURELETHEARNIATEARTIATHERIENOTIONDE, πu=364.40
 11 SHALLINTEREEQUISTREATHENINEVERITHEASHEATHESTORTHOMENNETHERIE, πu=382.99
 12 RTERSEADENESQUATEREATHERENEVERETHEATHEATHEARINCHEREATITHEANE, πu=386.22
 13 STAITHERE

: 